In [ ]:
# Імпорт бібліотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.optim as optim

# 1. Завантаження та попередня обробка даних
data = pd.read_csv('creditcard.csv')
print(data.head())

# Масштабування ознак
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data.drop('Class', axis=1))
X_train, X_test, y_train, y_test = train_test_split(data_scaled, data['Class'], test_size=0.2, random_state=42)

# 2. Метод Isolation Forest
model_if = IsolationForest(contamination=0.01, random_state=42)
y_pred_if = model_if.fit_predict(X_test)
y_pred_if = np.where(y_pred_if == -1, 1, 0)  # -1 означає аномалію
print("Метод Isolation Forest:")
print(classification_report(y_test, y_pred_if))

# 3. Метод One-Class SVM
model_svm = OneClassSVM(kernel='rbf', gamma=0.1, nu=0.01)
model_svm.fit(X_train)
y_pred_svm = model_svm.predict(X_test)
y_pred_svm = np.where(y_pred_svm == -1, 1, 0)
print("\nМетод One-Class SVM:")
print(classification_report(y_test, y_pred_svm))

# 4. Метод Local Outlier Factor 
model_lof = LocalOutlierFactor(n_neighbors=20, contamination=0.01)
y_pred_lof = model_lof.fit_predict(X_test)
y_pred_lof = np.where(y_pred_lof == -1, 1, 0)
print("\nМетод Local Outlier Factor:")
print(classification_report(y_test, y_pred_lof))

# 5. Автоенкодер на основі PyTorch
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
        )
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Параметри автоенкодера
input_dim = X_train.shape[1]
autoencoder = Autoencoder(input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
num_epochs = 10

# Навчання автоенкодера
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = autoencoder(torch.Tensor(X_train))
    loss = criterion(outputs, torch.Tensor(X_train))
    loss.backward()
    optimizer.step()

# Виявлення аномалій за допомогою автоенкодера
outputs = autoencoder(torch.Tensor(X_test))
losses = torch.mean((outputs - torch.Tensor(X_test)) ** 2, dim=1).detach().numpy()
threshold = np.percentile(losses, 99)  # Задаємо поріг як 99-й перцентиль
y_pred_autoencoder = np.where(losses > threshold, 1, 0)


print("\nАвтоенкодер:")
print(classification_report(y_test, y_pred_autoencoder))


Звіт про результати тестування методів виявлення аномалій
1. Найкращі результати
На основі метрики classification_report для кожного методу (наприклад, Precision, Recall, F1-score) ми можемо порівняти їхню продуктивність. Часто метод, який має вищий F1-score для класу аномалій (Class = 1), вважається найбільш ефективним для виявлення шахрайства, оскільки цей клас менш розповсюджений і важливий для точної класифікації.

2. Порівняння методів
    Isolation Forest:

Isolation Forest, який базується на випадкових деревах, часто демонструє хороші результати при виявленні аномалій у високовимірних даних. Цей метод інтуїтивно спрямований на виділення «аномальних» точок у просторах, де щільність даних значно нижча.
Для даного набору даних Isolation Forest може показувати сильний результат, оскільки метод добре працює з великими вибірками, зберігаючи ефективність.
    
    One-Class SVM:

One-Class SVM часто показує добрі результати в ситуаціях, коли дані важко розділити лінійно. У цьому випадку використання RBF kernel допомагає врахувати нелінійності в даних.
Проте метод може бути чутливим до вибору параметра nu (який вказує на очікувану частку аномалій), і це може впливати на результати на нових даних.
    
    Local Outlier Factor (LOF):

LOF показує хороші результати для виявлення локальних аномалій, оскільки враховує щільність сусідніх точок. Цей метод може бути ефективним для даних з локальними аномаліями, але може погано працювати з розрідженими даними.
У нашому випадку LOF може бути менш ефективним, якщо шахрайські транзакції мають тенденцію до створення нових, але загалом розріджених кластерів.
    
    Автоенкодер:

Глибокий нейронний автоенкодер вивчає стисле представлення даних, намагаючись відтворити вихідний вхід. Він може бути ефективним для аномалій, оскільки вивчає структуру даних і визначає елементи, які відхиляються від загальної схеми.
Для великого обсягу даних автоенкодери можуть потребувати більше часу для навчання. Однак, якщо структура аномалій значно відрізняється від основної частини даних, автоенкодер може точно ідентифікувати аномальні точки за рахунок великих відмінностей у відтворювальній помилці.
3. Аналіз продуктивності автоенкодера
Автоенкодер як глибока нейронна мережа має потенціал для ефективного виявлення аномалій, коли набір даних складний і містить нелінійні зв’язки. Для нашої задачі автоенкодер дозволяє виявити аномалії, які мають значно більші відмінності від загальної структури даних. Його використання може бути виправданим у випадках, коли ми маємо складні або важкодоступні особливості даних, які інші методи (на кшталт Isolation Forest чи LOF) не можуть повністю врахувати. Проте автоенкодери потребують більших обчислювальних ресурсів та часу на тренування, що може бути обмеженням для їх використання в реальному часі.

    Пояснення метрик
Precision показує, який відсоток транзакцій, позначених як аномальні, дійсно є шахрайськими.

Recall демонструє, який відсоток реальних шахрайських транзакцій було правильно позначено як аномальні.

F1-score – це зважене середнє між Precision та Recall, яке є особливо корисним, коли важливий баланс між точністю та повнотою (Recall).